In [7]:
import os, sys
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [8]:
# take the noted finetuned eval data
df_wwmlm_1e5 = pd.DataFrame({'task_name':['absa-airy', 'emotion-twitter', 'doc-sentiment-prosa', 'keyword-extraction-prosa', 'ner-grit', 'pos-idn', 'term-extraction-airy', 'entailment-ui', 'pos-prosa', 'ner-prosa'],
                              'F1':[0.7085, 0.7344, 0.8892, 0.6221, 0.7385, 0.9478, 0.8911, 0.7711, 0.964415, 0.789788]})
df_wwmlm_1e5['exp_name'] = 'babert_model_bpe_wwmlm_b8_step1_gamma0.8_lr1e-5_early10'
df_wwmlm_1e5['stats'] = 'mean'

df_wwmlm_625e6 = pd.DataFrame({'task_name':['absa-airy', 'emotion-twitter', 'doc-sentiment-prosa', 'keyword-extraction-prosa', 'ner-grit', 'pos-idn', 'term-extraction-airy', 'entailment-ui', 'pos-prosa', 'ner-prosa'],
                              'F1':[0.6781, 0.7233, 0.8602, 0.6447, 0.7483, 0.9602, 0.9039, 0.8219, 0.959804, 0.800779]})
df_wwmlm_625e6['exp_name'] = 'babert_model_bpe_wwmlm_b8_step1_gamma0.8_lr6.25e-6_early10'
df_wwmlm_625e6['stats'] = 'mean'

xlmr_base = pd.DataFrame({'task_name':['absa-airy', 'emotion-twitter', 'doc-sentiment-prosa', 'keyword-extraction-prosa', 'ner-grit', 'ner-prosa', 'pos-prosa', 'pos-idn', 'term-extraction-airy', 'entailment-ui'],
                            'F1':[0.7126, 0.7251, 0.9229, 0.6206, 0.8011, 0.7967, 0.9667, 0.9672, 0.8984, 0.8278]})
xlmr_base['exp_name'] = 'xlmr_base'
xlmr_base['stats'] = 'mean'

babert_bpe_mlm = pd.DataFrame({'task_name':['absa-airy', 'emotion-twitter', 'doc-sentiment-prosa', 'keyword-extraction-prosa', 'ner-grit', 'ner-prosa', 'pos-prosa', 'pos-idn', 'term-extraction-airy', 'entailment-ui'],
                                'F1':[0.7287, 0.7559, 0.8981, 0.6558, 0.7582, 0.8180, 0.9692, 0.9624, 0.9020, 0.8325]})
babert_bpe_mlm['exp_name'] = 'babert_bpe_mlm'
babert_bpe_mlm['stats'] = 'mean'

xlm_mlm_large = pd.DataFrame({'task_name':['absa-airy', 'emotion-twitter', 'doc-sentiment-prosa', 'keyword-extraction-prosa', 'ner-grit', 'ner-prosa', 'pos-prosa', 'pos-idn', 'term-extraction-airy', 'entailment-ui'],
                                'F1':[0.5849, 0.6546, 0.8567, 0.5703, 0.8138, 0.8169, 0.9677, 0.9685, 0, 0.6500]})
xlm_mlm_large['exp_name'] = 'xlm_mlm_large'
xlm_mlm_large['stats'] = 'mean'

xlmr_large = pd.DataFrame({'task_name':['absa-airy', 'emotion-twitter', 'doc-sentiment-prosa', 'keyword-extraction-prosa', 'ner-grit', 'ner-prosa', 'pos-prosa', 'pos-idn', 'term-extraction-airy', 'entailment-ui'],
                                'F1':[0.7500, 0.7677, 0.9335, 0.6092, 0.7939, 0, 0.0794, 0.9689, 0.9087, 0.8728]})
xlmr_large['exp_name'] = 'xlmr_large'
xlmr_large['stats'] = 'mean'

# take the recently finetuned eval data
base_path = './save'
dfs = []
for task_path in os.listdir(base_path):
    for exp_path in os.listdir(f'{base_path}/{task_path}'):
        if os.path.exists(f'{base_path}/{task_path}/{exp_path}/evaluation_result.csv'):
            df = pd.read_csv(f'{base_path}/{task_path}/{exp_path}/evaluation_result.csv')
            df.columns = ['stats'] + df.columns[1:].tolist()
            df['task_name'] = task_path
            df['exp_name'] = exp_path
            dfs.append(df.loc[df['stats'] == 'mean',:])
combined_df = pd.concat(dfs + [df_wwmlm_1e5, df_wwmlm_625e6, xlmr_base, babert_bpe_mlm, xlm_mlm_large, xlmr_large], sort=False)
combined_df.reset_index(drop=True, inplace=True)

for metric in ['ACC', 'F1', 'REC', 'PRE']:
    combined_df[metric] = combined_df[metric].apply(lambda x: x / 100 if x > 1 else x)

# Each Task's Standings vs NoN Large Models

In [9]:
analyzed_df = combined_df[~combined_df['exp_name'].isin(['xlmr_large', 'xlm_mlm_large'])]
maxes = analyzed_df.groupby('task_name')['F1'].idxmax()
combined_df_maxes = analyzed_df.loc[maxes][['exp_name', 'task_name', 'F1']]
undefeated_list = ['doc-sentiment-prosa', 'pos-idn', 'ner-grit']
combined_df_maxes[combined_df_maxes['task_name'].isin(undefeated_list)]

,exp_name,task_name,F1
213,xlmr_base,doc-sentiment-prosa,0.9229
215,xlmr_base,ner-grit,0.8011
218,xlmr_base,pos-idn,0.9672


In [10]:
combined_df_maxes[~combined_df_maxes['task_name'].isin(undefeated_list)]

,exp_name,task_name,F1
121,babert_model_word_mlm_b8_step1_gamma0.8_lr1e-5_early10,absa-airy,0.731188
222,babert_bpe_mlm,emotion-twitter,0.755900
230,babert_bpe_mlm,entailment-ui,0.832500
224,babert_bpe_mlm,keyword-extraction-prosa,0.655800
163,babert_model_bpe_wwmlm_128_256_ckpt_1000000_b8_step1_gamma0.8_lr1e-5_early5,ner-prosa,0.831198
227,babert_bpe_mlm,pos-prosa,0.969200
21,babert_model_bpe_wwmlm_ckpt_2000000_b8_step1_gamma0.8_lr1e-5_early10,qa-factoid-itb,0.442857
57,babert_model_bpe_wwmlm_128_256_ckpt_1000000_b8_step1_gamma0.8_lr1e-5_early5,term-extraction-airy,0.905271


If versus large model, we haven't yet defeated:

- __absa-airy__: __75__
- __doc-sentiment-prosa__: __93.35 or 92.29__
- __emotion-twitter__: __76.77__ (BASEnya kita udah menang)
- __entailment-ui__: __87.28__ (BASEnya kita udah menang) -> cepet finetuningnya
- __ner-grit__: __81.38__
- __pos-idn__: __96.89 or 96.72__
- __term-extraction-airy__: __90.87__ (BASEnya kita 90.20)

# IndoNLU Eval

In [11]:
indonlu_eval = combined_df[~combined_df['task_name'].isin(['qa-factoid-itb'])]
indonlu_eval_grouped = indonlu_eval.groupby('exp_name').agg(['mean', 'count'])
indonlu_eval_grouped[indonlu_eval_grouped[('F1','count')]==10][['F1']].reset_index().sort_values(by=('F1','mean'), ascending=False).reset_index(drop=True)

exp_name  \
                                                                                      
0   babert_bpe_mlm                                                                    
1   xlmr_base                                                                         
2   babert_model_bpe_wwmlm_128_256_ckpt_1000000_b8_step1_gamma0.8_lr1e-5_early10      
3   babert_model_bpe_wwmlm_128_256_ckpt_1000000_b8_step1_gamma0.8_lr1e-5_early5       
4   babert_model_bpe_wwmlm_128_256_ckpt_1000000_b8_step1_gamma0.8_lr6.25e-6_early10   
5   babert_model_bpe_wwmlm_ckpt_2000000_b8_step1_gamma0.8_lr1e-5_early10              
6   babert_model_bpe_wwmlm_ckpt_2000000_b8_step1_gamma0.8_lr1e-5_early5               
7   babert_model_bpe_wwmlm_b8_step1_gamma0.8_lr6.25e-6_early10                        
8   babert_model_bpe_wwmlm_b8_step1_gamma0.8_lr1e-5_early10                           
9   babert_model_word_mlm_b8_step1_gamma0.8_lr1e-5_early5                             
10  babert_model_word_mlm_b8_step1_gamma0.8_lr1e-5_early10                            
11  babert_model_unigram_mlm_b8_step1_gamma0.8_lr1e-5_early10                         
12  babert_model_unigram_mlm_b8_step1_gamma0.8_lr1e-5_early5                          
13  babert_model_word_mlm_512_ckpt_1150000_b8_step1_gamma0.8_lr1e-5_early5            
14  babert_model_unigram_mlm_512_ckpt_160000_b8_step1_gamma0.8_lr1e-5_early5          
15  babert_model_bpe_mlm_lr_b8_step1_gamma0.8_lr1e-5_early5                           
16  babert_model_unigram_mlm_512_ckpt_132500_b8_step1_gamma0.8_lr1e-5_early5          
17  babert_model_bpe_512_full_lr2e5_ckpt_220000_b8_step1_gamma0.8_lr1e-5_early5       
18  babert_model_bpe_512_full_lr2e5_ckpt_185000_b8_step1_gamma0.8_lr1e-5_early5       
19  babert_model_unigram_mlm_small_bs_ckpt_160000_b8_step1_gamma0.8_lr1e-5_early5     
20  babert_model_unigram_mlm_small_bs_ckpt_130000_b8_step1_gamma0.8_lr1e-5_early5     
21  xlm_mlm_large                                                                     
22  xlmr_large                                                                        

          F1        
        mean count  
0   0.828080  10    
1   0.823910  10    
2   0.818826  10    
3   0.816919  10    
4   0.813624  10    
5   0.811920  10    
6   0.811793  10    
7   0.810118  10    
8   0.805690  10    
9   0.796764  10    
10  0.796764  10    
11  0.796192  10    
12  0.795271  10    
13  0.793463  10    
14  0.791164  10    
15  0.789484  10    
16  0.788199  10    
17  0.779104  10    
18  0.778346  10    
19  0.776894  10    
20  0.768803  10    
21  0.688340  10    
22  0.668410  10

In [12]:
indonlu_eval = combined_df[~combined_df['task_name'].isin(['qa-factoid-itb', 'term-extraction-airy', 'pos-prosa'])]
indonlu_eval_grouped = indonlu_eval.groupby('exp_name').agg(['mean', 'count'])
indonlu_eval_grouped[indonlu_eval_grouped[('F1','count')]==8][['F1']].reset_index().sort_values(by=('F1','mean'), ascending=False)

exp_name  \
                                                                                      
0   babert_bpe_mlm                                                                    
21  xlmr_base                                                                         
4   babert_model_bpe_wwmlm_128_256_ckpt_1000000_b8_step1_gamma0.8_lr1e-5_early10      
5   babert_model_bpe_wwmlm_128_256_ckpt_1000000_b8_step1_gamma0.8_lr1e-5_early5       
6   babert_model_bpe_wwmlm_128_256_ckpt_1000000_b8_step1_gamma0.8_lr6.25e-6_early10   
9   babert_model_bpe_wwmlm_ckpt_2000000_b8_step1_gamma0.8_lr1e-5_early10              
10  babert_model_bpe_wwmlm_ckpt_2000000_b8_step1_gamma0.8_lr1e-5_early5               
8   babert_model_bpe_wwmlm_b8_step1_gamma0.8_lr6.25e-6_early10                        
7   babert_model_bpe_wwmlm_b8_step1_gamma0.8_lr1e-5_early10                           
19  babert_model_word_mlm_b8_step1_gamma0.8_lr1e-5_early5                             
18  babert_model_word_mlm_b8_step1_gamma0.8_lr1e-5_early10                            
13  babert_model_unigram_mlm_b8_step1_gamma0.8_lr1e-5_early10                         
14  babert_model_unigram_mlm_b8_step1_gamma0.8_lr1e-5_early5                          
17  babert_model_word_mlm_512_ckpt_1150000_b8_step1_gamma0.8_lr1e-5_early5            
12  babert_model_unigram_mlm_512_ckpt_160000_b8_step1_gamma0.8_lr1e-5_early5          
11  babert_model_unigram_mlm_512_ckpt_132500_b8_step1_gamma0.8_lr1e-5_early5          
3   babert_model_bpe_mlm_lr_b8_step1_gamma0.8_lr1e-5_early5                           
2   babert_model_bpe_512_full_lr2e5_ckpt_220000_b8_step1_gamma0.8_lr1e-5_early5       
1   babert_model_bpe_512_full_lr2e5_ckpt_185000_b8_step1_gamma0.8_lr1e-5_early5       
16  babert_model_unigram_mlm_small_bs_ckpt_160000_b8_step1_gamma0.8_lr1e-5_early5     
20  xlm_mlm_large                                                                     
15  babert_model_unigram_mlm_small_bs_ckpt_130000_b8_step1_gamma0.8_lr1e-5_early5     
22  xlmr_large                                                                        

          F1        
        mean count  
0   0.801200  8     
21  0.796750  8     
4   0.789828  8     
5   0.787445  8     
6   0.784566  8     
9   0.781867  8     
10  0.781867  8     
8   0.779685  8     
7   0.775173  8     
19  0.766666  8     
18  0.766666  8     
13  0.764182  8     
14  0.763030  8     
17  0.762454  8     
12  0.758065  8     
11  0.754895  8     
3   0.754626  8     
2   0.743777  8     
1   0.742356  8     
16  0.740677  8     
20  0.739463  8     
15  0.729813  8     
22  0.712000  8

In [ ]:
sorted_model_list = [
    'scratch',
    'word2vec_',
    'fasttext_',
    'fasttext-cc-id',
    'fasttext-cc-id-no-oov',
    'babert-opensubtitle',
    'babert-opensubtitle',
    'bert-base-multilingual-uncased',
    'xlm-roberta-base'
]

In [ ]:
def exp_to_index(exp_name):
    for i, model in enumerate(sorted_model_list):
        if model in exp_name:
            return i
combined_df['model_index'] = combined_df['exp_name'].apply(lambda x: exp_to_index(x))

In [ ]:
combined_df = combined_df.sort_values(['task_name', 'model_index'])

In [ ]:
combined_df.to_csv('aggregated_result.csv', index=False)